<a href="https://colab.research.google.com/github/poltorashka-s-BMa/course-os-linux/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22var2_Malysheva_Ekaterina_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def arr_1(n, arr):
    left = 0
    for j in range(n):
        if arr[j] != 0:
            arr[left] = arr[j]
            left += 1

    for left in range(left, n):
        arr[left] = 0


    return arr


In [ ]:
n = 6
arr = [0, 0, 1, 0, 3, 12]
print(arr_1(n, arr))

[1, 3, 12, 0, 0, 0]


In [ ]:
def remove_element(n, element, arr):
    write_ptr = 0
    for num in arr:
        if num != element:
            arr[write_ptr] = num
            write_ptr += 1
    return arr[:write_ptr]

In [ ]:
!gdown 1Lo47fXx1RrJG3v9E-Gck1T45n5bJ_SJf

Downloading...
From: https://drive.google.com/uc?id=1Lo47fXx1RrJG3v9E-Gck1T45n5bJ_SJf
To: /content/data.adult.csv
100% 1.54M/1.54M [00:00<00:00, 112MB/s]


In [ ]:
# -*- coding: utf-8 -*-
"""var2_Malysheva_Ekaterina.ipynb"

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1duXSZaUCw31_nnt07pdfDXQSrYbPCNAm
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Загрузка данных
data = pd.read_csv('data.adult.csv')

# Просмотр первых строк
print(data.head())
print("\nИнформация о данных:")
print(data.info())

# Замена "?" на NaN и удаление пропусков
data = data.replace('?', pd.NA).dropna()

# Разделение на обучающую и тестовую выборки
train_data, test_data = train_test_split(data, test_size=0.4, random_state=42)

# Преобразование целевой переменной
train_data['income'] = train_data['>50K,<=50K'].apply(lambda x: 1 if x.strip() == '>50K' else 0)
test_data['income'] = test_data['>50K,<=50K'].apply(lambda x: 1 if x.strip() == '>50K' else 0)

# Удаление исходной целевой переменной из данных
train_data = train_data.drop('>50K,<=50K', axis=1)
test_data = test_data.drop('>50K,<=50K', axis=1)

# One-hot encoding для категориальных признаков
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation',
                   'relationship', 'race', 'sex']
train_data = pd.get_dummies(train_data, columns=categorical_cols)
test_data = pd.get_dummies(test_data, columns=categorical_cols)

print("Распределение классов в обучающей выборке:")
print(train_data['income'].value_counts(normalize=True))

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Выделение признаков и целевой переменной
X_train = train_data.drop('income', axis=1)
y_train = train_data['income']

# Подбор оптимальной глубины дерева
param_grid = {'max_depth': range(1, 21)}
tree = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(tree, param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)

# Лучшие параметры
print(f"Лучшая глубина дерева: {grid_search.best_params_['max_depth']}")
print(f"Лучшая точность: {grid_search.best_score_:.3f}")

from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Подбор количества деревьев
n_estimators = [10, 50, 100, 200, 300, 400, 500]
param_grid = {'n_estimators': n_estimators}
forest = RandomForestClassifier(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(forest, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Лучшее количество деревьев
best_n = grid_search.best_params_['n_estimators']
print(f"Оптимальное количество деревьев: {best_n}")

# Подбор других параметров для Random Forest
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2', None]
}
forest = RandomForestClassifier(n_estimators=best_n, random_state=42, n_jobs=-1)
grid_search = GridSearchCV(forest, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Лучшие параметры
print("Лучшие параметры для Random Forest:")
print(grid_search.best_params_)

from sklearn.metrics import accuracy_score, classification_report

# Подготовка тестовых данных
X_test = test_data.drop('income', axis=1)
y_test = test_data['income']

# Лучшие модели
best_tree = DecisionTreeClassifier(max_depth=grid_search.best_params_['max_depth'], random_state=42)
best_forest = RandomForestClassifier(n_estimators=best_n, **grid_search.best_params_, random_state=42, n_jobs=-1)

# Обучение и предсказание
best_tree.fit(X_train, y_train)
best_forest.fit(X_train, y_train)

tree_pred = best_tree.predict(X_test)
forest_pred = best_forest.predict(X_test)

# Оценка качества
print("Decision Tree:")
print(classification_report(y_test, tree_pred))
print("Accuracy:", accuracy_score(y_test, tree_pred))

print("\nRandom Forest:")
print(classification_report(y_test, forest_pred))
print("Accuracy:", accuracy_score(y_test, forest_pred))

   age  workclass  fnlwgt     education  education-num      marital-status  \
0   34  Local-gov  284843       HS-grad              9       Never-married   
1   40    Private  190290  Some-college             10            Divorced   
2   36  Local-gov  177858     Bachelors             13  Married-civ-spouse   
3   22    Private  184756  Some-college             10       Never-married   
4   47    Private  149700     Bachelors             13  Married-civ-spouse   

        occupation   relationship   race     sex  capital-gain  capital-loss  \
0  Farming-fishing  Not-in-family  Black    Male           594             0   
1            Sales  Not-in-family  White    Male             0             0   
2   Prof-specialty      Own-child  White    Male             0             0   
3            Sales      Own-child  White  Female             0             0   
4     Tech-support        Husband  White    Male         15024             0   

   hours-per-week >50K,<=50K  
0              60  